In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import warnings
import os

# Scikit-learn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# XGBoost
import xgboost as xgb
# Cấu hình hiển thị
pd.set_option('display.float_format', lambda x: '%.4f' % x)
warnings.filterwarnings('ignore')
# Kiểm tra GPU
import tensorflow as tf
print("🔥 GPU Status:", tf.config.list_physical_devices('GPU'))
print("Đã import thư viện thành công!")

🔥 GPU Status: []
Đã import thư viện thành công!


In [2]:
from sklearn.preprocessing import RobustScaler
# Đường dẫn file mới
DATA_PATH = '../data/processed/clean_vn_housing.csv'

df = pd.read_csv(DATA_PATH)
print(f"📂 Đã load dữ liệu: {df.shape}")

# 1. Tách Feature & Target
target_col = 'Total_Price_Billion'

# Loại bỏ các cột không dùng để train
# Lưu ý: File sạch đã bỏ Address và Price_per_m2, nhưng ta cứ drop lần nữa cho chắc
drop_cols = [target_col, 'Price_per_m2', 'Unnamed: 0', 'Địa chỉ']
X = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

# 2. Log Transform Target
# BẮT BUỘC: Giúp model dự đoán tốt dải giá rộng từ 1 tỷ đến 100 tỷ
y = np.log1p(df[target_col])

# 3. Chia tập Train/Test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Scaling (RobustScaler)
# Cực quan trọng khi có cột Length/Width/Area biên độ chênh lệch lớn
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Lưu Scaler cho App
if not os.path.exists('../models'): os.makedirs('../models')
joblib.dump(scaler, '../models/scaler_final.pkl')

print(f"✅ Dữ liệu đã chuẩn hóa. Số Features: {X_train.shape[1]}")

📂 Đã load dữ liệu: (65468, 246)
✅ Dữ liệu đã chuẩn hóa. Số Features: 245


In [3]:
def danh_gia_hieu_nang(model, X_test, y_test_log, ten_mo_hinh):
    # Dự đoán
    y_pred_log = model.predict(X_test)

    # Chuyển ngược Log -> Giá thực (Tỷ VNĐ)
    y_true = np.expm1(y_test_log)
    y_pred = np.expm1(y_pred_log)
    y_pred = np.maximum(y_pred, 0) # Chặn số âm

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print(f"\n{'='*40}")
    print(f"📊 KẾT QUẢ: {ten_mo_hinh.upper()}")
    print(f"{'='*40}")
    print(f"MAE  : {mae:.4f} Tỷ")
    print(f"RMSE : {rmse:.4f} Tỷ")
    print(f"R2   : {r2:.4f} ({r2*100:.2f}%)")

    if r2 > 0.7: print("   🚀 Model đã đạt mức độ chính xác cao!")
    elif r2 > 0.8: print("   🏆 Model xuất sắc (SOTA)!")

    return {'Model': ten_mo_hinh, 'MAE': mae, 'RMSE': rmse, 'R2': r2}

In [4]:
import time
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

print(">>> 1. Tuning Random Forest....")
t0 = time.time()

rf = RandomForestRegressor(random_state=42, n_jobs=-1)

param_grid_rf = {
    # Tăng số cây để "bù đắp" thông tin thiếu hụt bằng số đông
    'n_estimators': [500],

    # Tăng độ sâu: Vì chỉ biết đến Phường, cây cần sâu hơn để phân biệt
    # nhà to/nhà nhỏ trong cùng 1 phường.
    'max_depth': [30, 35],

    # max_features: Giữ 0.5 để nhìn bao quát các cột Phường (One-Hot)
    'max_features': [0.5, 'sqrt'],

    # Leaf = 2: Vẫn giữ quy tắc an toàn, không học thuộc lòng (Overfitting)
    # Vì không có tên đường, nếu để leaf=1 dễ bị nhiễu bởi các căn nhà cá biệt.
    'min_samples_leaf': [2],
    'min_samples_split': [5]
}

grid_rf = GridSearchCV(rf, param_grid_rf, cv=3, scoring='r2', verbose=1, n_jobs=-1)
grid_rf.fit(X_train_scaled, y_train)

best_rf = grid_rf.best_estimator_
print(f"⏱️ RF Time: {time.time() - t0:.1f}s")
print("Best Params RF:", grid_rf.best_params_)

joblib.dump(best_rf, '../models/rf_final.pkl')
res_rf = danh_gia_hieu_nang(best_rf, X_test_scaled, y_test, "Random Forest")

>>> 1. Tuning Random Forest....
Fitting 3 folds for each of 4 candidates, totalling 12 fits
⏱️ RF Time: 272.6s
Best Params RF: {'max_depth': 30, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 500}

📊 KẾT QUẢ: RANDOM FOREST
MAE  : 0.6182 Tỷ
RMSE : 0.9996 Tỷ
R2   : 0.8392 (83.92%)
   🚀 Model đã đạt mức độ chính xác cao!


In [5]:
train_r2 = r2_score(y_train, best_rf.predict(X_train_scaled))
test_r2  = r2_score(y_test, best_rf.predict(X_test_scaled))

print("Train R2:", train_r2)
print("Test  R2:", test_r2)
print("Gap     :", train_r2 - test_r2)

Train R2: 0.8901005038478998
Test  R2: 0.8088834543699078
Gap     : 0.08121704947799202


In [6]:
import xgboost as xgb

print(">>> 2. Tuning XGBoost....")
t0 = time.time()

xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist',
    device='cuda',
    random_state=42,
    n_jobs=1
)

param_grid_xgb = {
    # Cần nhiều cây hơn để hội tụ vì thiếu đặc trưng mạnh (Street)
    'n_estimators': [3000, 4000],

    # Học chậm để dò tìm quy luật tinh tế giữa Diện tích và Phường
    'learning_rate': [0.01],

    # Độ sâu: 10-12. Cần đủ sâu để model hiểu: "Phường A + Mặt tiền 5m + Diện tích 50m = Giá cao"
    'max_depth': [10, 12],

    # Regularization vừa phải. Không phạt quá mạnh (như khi có Street) vì dữ liệu đang thiếu thông tin.
    'reg_alpha': [0.1, 0.5],    # Giảm nhẹ so với lúc có Street
    'reg_lambda': [2, 5],

    'subsample': [0.85],
    'colsample_bytree': [0.6]   # Nhìn 60% số cột (chủ yếu là các cột Phường)
}

grid_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=3, scoring='r2', verbose=1, n_jobs=1)
grid_xgb.fit(X_train_scaled, y_train)

best_xgb = grid_xgb.best_estimator_
print(f"⏱️ XGB Time: {time.time() - t0:.1f}s")
print("Best Params XGB:", grid_xgb.best_params_)

joblib.dump(best_xgb, '../models/xgb_final.pkl')
res_xgb = danh_gia_hieu_nang(best_xgb, X_test_scaled, y_test, "XGBoost")

>>> 2. Tuning XGBoost....
Fitting 3 folds for each of 16 candidates, totalling 48 fits
⏱️ XGB Time: 2484.5s
Best Params XGB: {'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 12, 'n_estimators': 3000, 'reg_alpha': 0.1, 'reg_lambda': 5, 'subsample': 0.85}

📊 KẾT QUẢ: XGBOOST
MAE  : 0.6009 Tỷ
RMSE : 0.9981 Tỷ
R2   : 0.8397 (83.97%)
   🚀 Model đã đạt mức độ chính xác cao!


In [7]:
train_r2 = r2_score(y_train, best_xgb.predict(X_train_scaled))
test_r2  = r2_score(y_test, best_xgb.predict(X_test_scaled))

print("Train R2:", train_r2)
print("Test  R2:", test_r2)
print("Gap     :", train_r2 - test_r2)

Train R2: 0.8796989344009613
Test  R2: 0.8181162344175203
Gap     : 0.0615826999834409


In [8]:
import pandas as pd

# 1. Gom kết quả từ 3 model (Đảm bảo bạn đã chạy xong 3 cell trên)
results_list = [res_rf, res_xgb]

# 2. Tạo bảng DataFrame
df_results = pd.DataFrame(results_list)

# 3. Sắp xếp theo R2 giảm dần (Model xịn nhất lên đầu)
df_results = df_results.sort_values(by='R2', ascending=False).reset_index(drop=True)

# 4. Hiển thị bảng đơn giản
print(">>> BẢNG SO SÁNH KẾT QUẢ CUỐI CÙNG")
display(df_results)

# (Tùy chọn) Lưu ra file Excel/CSV để nộp báo cáo
df_results.to_csv('../data/processed/ket_qua_so_sanh.csv', index=False)

>>> BẢNG SO SÁNH KẾT QUẢ CUỐI CÙNG


,Model,MAE,RMSE,R2
0,XGBoost,0.6009,0.9981,0.8397
1,Random Forest,0.6182,0.9996,0.8392
